In [86]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [126]:
path = "../hand_labeled/"
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [88]:
191*120/3600

6.366666666666666

In [89]:
# 文章ごとに n-gram を考えてみる
def get_ngram_set(doc, N=3):
    if isinstance(doc, str):
        doc = nlp(doc)
    surfaces = [token.text for token in doc]
    ngram_set = set()
    filled = ["FOS", *surfaces, "EOS"]
    # print(filled)
    for i in range(len(filled)-N+1):
        f = "_".join(filled[i:i+N])
        ngram_set.add(f)
    return ngram_set

In [90]:
def check_repeat_rate(target:set, history:list, border=0.7):
    t_list = list(target)
    for prev_set in history:
        size = len(prev_set)
        hit = 0
        for t in t_list:
            if t in prev_set:
                hit+=1
        if hit/size >= border:
            return True
    
    return False

In [127]:
error = "Repetition"
y = []
for conv in convs:
    for ut in conv:
        if not ut.is_system():
            continue
        # 
        if ut.is_exist_error():
            if ut.is_error_included(error):
                # print(ut.errors)
                y.append(1)
            else:
                y.append(0)

In [128]:
len(y)

1386

In [93]:
idf_path = "../../corpus/wiki/idf_wiki_v2.json"
with open(idf_path, "r") as f:
    idf_dict = json.load(f)

In [94]:
def is_under_idf_border_morpheme(text, idf_border):
    for token in mecab_tokenize(text):
        key = token
        if key in idf_dict:
            if idf_dict[key] > idf_border:
                return False
        else:
            return False
    return True

In [95]:
def check_repeat_rate_ret_i(target:set, history:list, border=0.7):
    t_list = list(target)
    for i, prev_set in enumerate(history):
        size = len(prev_set)
        hit = 0
        for t in t_list:
            if t in prev_set:
                hit+=1
        if hit/size >= border:
            return i
    
    return -1

In [129]:
import Levenshtein

y_pred = []
border = 0.8

border_10 = 14.62592894814336
border_1 = 9.88253708481772
border_005 = 5.340630805533431

pair_list = []

for conv in tqdm(convs):
    ngram_sets = []
    prev_sents = []
    for ut in conv:
        if not ut.is_system():
            continue
        utt = ut.utt
        doc = nlp(utt)
        if ut.is_exist_error():
            y_pred.append(0)
        
        for sent in doc.sents:
            # idf が小さい場合は，スルーしましょう
            if is_under_idf_border_morpheme(sent.orth_, border_005):
                # print(sent)
                continue
            ngram_set = get_ngram_set(sent, N=3)
            # これまでのセットで重複が大きいものがあるかチェック！
            if ut.is_exist_error():
                ngram_checked = check_repeat_rate_ret_i(target=ngram_set, history=ngram_sets, border=0.8)
                # if check_repeat_rate_ret_i(target=ngram_set, history=ngram_sets, border=0.8):
                    # print(ut, ut.errors)
                if ngram_checked >= 0:
                    y_pred[-1] = 1
                    pair_list.append([sent.orth_, prev_sents[ngram_checked], "ngram"])

                if y_pred[-1] == 0:
                    for i, prev in enumerate( prev_sents ) :
                        if Levenshtein.ratio(sent.orth_, prev) >= border:
                            pair_list.append([sent.orth_, prev_sents[i], "leven"])
                            y_pred[-1] = 1
                

            ngram_sets.append(ngram_set)
            prev_sents.append(sent.orth_)

100%|██████████| 200/200 [00:26<00:00,  7.59it/s]


In [130]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[1296   34]
 [  10   46]]
accuracy =  0.9682539682539683
precision =  0.575
recall =  0.8214285714285714
f1 score =  0.676470588235294


- ngram  border 0.8

        confusion matrix = 
        [[2131   21]
        [  29   19]]
        accuracy =  0.9772727272727273
        precision =  0.475
        recall =  0.3958333333333333
        f1 score =  0.4318181818181817

    - 再現率が欲しい

- Levenshtein bordr 0.8
    
        confusion matrix = 
        [[2102   50]
        [  19   29]]
        accuracy =  0.9686363636363636
        precision =  0.3670886075949367
        recall =  0.6041666666666666
        f1 score =  0.45669291338582674

    - 悪くないが，適合率が低い

- ngram  border 0.8 and Levenshtein bordr 0.8 and idf 005%
    - 仮定 : 「そうですよね」のように，全く情報量がない発話が繰り返された場合は不要な繰り返しの可能性が低い
        - idf 辞書の活用
    - 予想 : 誤検出が減る
    
            confusion matrix = 
            [[2091   61]
            [  15   33]]
            accuracy =  0.9654545454545455
            precision =  0.35106382978723405
            recall =  0.6875
            f1 score =  0.46478873239436624


    - 評価データでの実行
            
            confusion matrix = 
            [[2080   64]
            [  10   46]]
            accuracy =  0.9663636363636363
            precision =  0.41818181818181815
            recall =  0.8214285714285714
            f1 score =  0.5542168674698795


    - 誤検出が増えたが，未検出が減った
        - len(sent) <3 の中に，情報量があり，似た発話があるが不要ではない
        - len(sent) <3 の中に検出するべきエラーが存在した
    
    - 今後
        - どのような発話が雑談において不要ではないのかの検討が出来ていない
            - 共起辞書が活用できるか？
                - データを見た限り，そう簡単にはいかない
        - 非常に近い距離で，動詞や形容詞だけが違うという場合が検出出来ていない
            - 検出された発話中で，名詞や動詞などを比較するべきか

In [59]:
i = 0
j = 0
for conv in convs:
    conv_list = []
    for ut in conv:
        conv_list.append(ut.utt)
        if not ut.is_system():
            continue
        # 本来エラーではないけどエラー扱い
        if ut.is_exist_error():
            if y[i]==0 and y_pred[i]==1:
                print("本来エラーではないけどエラー扱い", ut.did)
                print(pair_list[j])
                print()
            # if y[i]==1 and y_pred[i]==0:
            #     print("本来エラーなのに非エラー扱い", ut.did)
            #     print(conv_list)
            #     print()
            if y[i]==1 and y_pred[i]==1:
                print("よく検出した！えらいぞ", ut.did)
                print(conv_list)
                print(pair_list[j])
                print()
            
            if y_pred[i]==1:
                j += 1
            
            i+=1

よく検出した！えらいぞ 1470646494
['こんばんは！海へ行きたいね。', 'そうですね。最近とても暑いですから。', '海に行かないんですね？', '休日に行きたいと思います。', '海は大丈夫ですね', 'はい。あなたは海に行きますか？', '海は危険です', '何故ですか？', '海はいけないです', 'そうですか。山に行くのはどうでしょうか？', 'わあ！いいですね！', '山はお好きなのですか？', 'わあ！いいですね！']
['わあ！', 'わあ！', 'ngram']

よく検出した！えらいぞ 1470623274
['こんにちは。夏といえばスイカだね。', '美味しいですね。', '美味しいよ', 'でも高いですね', 'いくら', '1000円ぐらいかな？', 'スイカは実がなってたことあります', '意味が分かりません', 'あなたの好きな映画について教えてください', 'サスペンスが好きです', '映画はヒューマン系が好きですね', '映画の題名は？', '映画にはまるのです', '好きな俳優さんはいますか？', '映画が楽しみです', 'そうですか', 'あはい', '他に好きなものは？', '映画が楽しみですね']
['いいですね！', 'いいですね！', 'ngram']

本来エラーではないけどエラー扱い 1470624021
['映画が楽しみですね', '映画が楽しみです', 'leven']

本来エラーではないけどエラー扱い 1470624021
['ラーメンは美味しいですね', 'ラーメンはスープが美味しいですね', 'leven']

よく検出した！えらいぞ 1502868470
['こんにちは！海へ行きたいね。', '海いいですね', '気持ちがいいですね', '海いいですね！大好きです。', '海は好きですね', 'どこの海がお勧めですか？', '海は楽しいですね', 'でも私は泳げないんです。', '海は大好きですね', 'そういえば、山はどうですか？', '海はサイコーですね', '海外の海もいいものですよね', '海はキレーですね', '魚もおいしいですよね', '魚はおいんですか？？海は素晴らしいですね', '船旅もいいらしいですね', '海は素晴らしいですね']
['麺が美味しいですね', '麺が美味しい

In [60]:
# 共起辞書の活用
ppmi_dataname = "../../corpus/collocation/ppmi_ntt1"
ppmi_matrix = np.load(ppmi_dataname+".npy")

In [61]:
dictname = "../../corpus/collocation/word_dict/ppmi_word_dict_ntt1.json"
with open(dictname, "r") as f:
    word_dict = json.load(f)

In [62]:
def ppmi(ppmi_matrix, word_dict, x, y):
    if x not in word_dict.keys():
        return 0
    elif y not in word_dict.keys():
        return 0

    x_id = word_dict[x]
    y_id = word_dict[y]
    return ppmi_matrix[x_id, y_id]

In [63]:
ppmi(ppmi_matrix, word_dict, "祭り", "掛け声")

0

In [64]:
def colocate_rate(utt1, utt2):
    left_1 = filtering( nlp(utt1) , toyoshima_set)
    left_2 = filtering( nlp(utt2) , toyoshima_set)
    
    # 名詞がないよ
    if left_1[0] == "[NONE]" or  left_2[0] == "[NONE]":
        return 0

    # print(left_1, left_2)
    rate = 0
    for lx in left_1:
        for ly in left_2:
            rate += ppmi(ppmi_matrix, word_dict, lx, ly)
    rate = rate/(len(left_1)+len(left_2))
    return rate

In [65]:
from gensim.models import KeyedVectors
w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
# w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[1489] 2022-01-16 11:57:28,977 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[1489] 2022-01-16 11:58:33,262 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-16T11:58:33.262489', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [66]:
toyoshima_set = set("NOUN PROPN VERB ADJ".split())

def filtering(doc, filter_set):
    left = []
    for token in doc:
        if token.pos_ in filter_set:
            left.append(token.lemma_)
    return left if len(left)>0 else ["[NONE]"]

In [67]:
def w2v(word, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    if word in SYMBOL_w2v:
        vector = SYMBOL_w2v[word]
    elif word in w2v_model:
        vector = w2v_model[word]
    else:
        vector = SYMBOL_w2v["[UNK]"]
    return vector

In [68]:
wsv_dim = w2v_model["あ"].shape[0]
add_keys = ["FOS", "EOS", "[SEP]", "[UNK]", "[NONE]"]
add_weights = [np.random.randn(wsv_dim) for _ in range(len(add_keys))]
add_weights = [ v/np.linalg.norm(v) for v in add_weights ]
SYMBOL_w2v = dict(zip(add_keys, add_weights))

In [69]:
def doc2vec2(doc, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    left = filtering(doc, independent_set)
    return np.mean([ w2v(w, w2v_model, SYMBOL_w2v) for w in left], axis=0)

In [113]:
def sentence2formated(sen, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    docs = sentence2docs(sen, sents_span=False)
    vector = []
    for i, doc in enumerate(docs):
        if i==0:
            prev_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
        else:
            current_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
            diff_vec = np.abs(prev_vector-current_vector)       
            vector.append( diff_vec)
    return vector

In [71]:
def sentence2vec_diff(sen, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    docs = sentence2docs(sen, sents_span=False)
    vector = []
    for i, doc in enumerate(docs):
        if i==0:
            # prev_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
            prev_left= set( filtering(doc, independent_set) ) 
        else:
            # current_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
            currtnt_left = set ( filtering(doc, independent_set) )   
            prev_diff = currtnt_left - prev_left
            current_diff = prev_left - currtnt_left
            if len(prev_diff)==0:
                prev_v = 0
            else:
                prev_v = np.mean([ w2v(w, w2v_model, SYMBOL_w2v) for w in list(prev_diff)], axis=0)
            if len(current_diff)==0:
                current_v = 0
            else:
                current_v = np.mean([ w2v(w, w2v_model, SYMBOL_w2v) for w in list(current_diff)], axis=0)
            vector.append( np.abs(prev_v-current_v)   )
    return vector
    

In [72]:
def f1_score_plain(test, pred):
    return f1_score(y_true=test, y_pred=pred)

- eval border : 3.1
- train border : 4.15

In [116]:
len(pair_list)

144

In [131]:
border_w2v = 2.3
max_f1 = 0
epoch = 50
gamma = 0.05
for k in tqdm(range(epoch)):

    i = 0
    j = 0
    y_pred2 = []
    for conv in convs:
        conv_list = []
        for ut in conv:
            conv_list.append(ut.utt)
            if not ut.is_system():
                continue
            # 本来エラーではないけどエラー扱い
            if ut.is_exist_error():
                if y_pred[i]==1:
                    # ペアをベクトル化し，差分を計算
                    # print(pair_list[j][:2])
                    vector = sentence2vec_diff(pair_list[j][:2], w2v_model, SYMBOL_w2v)[0]
                    # print(np.linalg.norm(vector))
                    if np.linalg.norm(vector) > border_w2v:
                        y_pred2.append(0)
                        # if y[i] == 1:
                        #     print( pair_list[j][:2] ) 
                    else:
                        y_pred2.append(1)
                    j += 1
                else:
                    y_pred2.append(0)
                
                i+=1
    f1 = f1_score_plain(y, y_pred2)
    # print("f1 score:{0}, border:{1}".format(max_f1, border_w2v))
    if f1 > max_f1:
        max_f1 = f1
        print("f1 score:{0}, border:{1}".format(max_f1, border_w2v))
    border_w2v += gamma


  2%|▏         | 1/50 [00:01<00:52,  1.07s/it]

f1 score:0.6923076923076924, border:2.3


 12%|█▏        | 6/50 [00:06<00:45,  1.03s/it]

f1 score:0.7102803738317757, border:2.549999999999999


 22%|██▏       | 11/50 [00:11<00:40,  1.04s/it]

f1 score:0.7222222222222223, border:2.799999999999998


 34%|███▍      | 17/50 [00:17<00:34,  1.03s/it]

f1 score:0.7272727272727273, border:3.099999999999997


100%|██████████| 50/50 [00:51<00:00,  1.03s/it]


In [136]:
i = 0
j = 0
y_pred2 = []
for conv in convs:
    conv_list = []
    for ut in conv:
        conv_list.append(ut.utt)
        if not ut.is_system():
            continue
            # 本来エラーではないけどエラー扱い
        if ut.is_exist_error():
            if y_pred[i]==1:
                    # ペアをベクトル化し，差分を計算
                    # print(pair_list[j][:2])
                vector = sentence2vec_diff(pair_list[j][:2], w2v_model, SYMBOL_w2v)[0]
                    # print(np.linalg.norm(vector))
                if np.linalg.norm(vector) > 4.15:
                    y_pred2.append(0)
                    if y[i] == 0:
                        print( pair_list[j][:2] ) 
                else:
                    y_pred2.append(1)
                j += 1
            else:
                y_pred2.append(0)
                
            i+=1


['天ぷらは食べるのが好きですね', '天ぷらは食べるのが美味しいですね']
['かけ声をだしたことありますね', 'かけ声を覚えたことあります']
['かけ声をあげるかもしれない', 'かけ声を覚えるかもしれない']
['映画は音楽がいいですね', '映画は一人がいいですね']
['仲間由紀恵は美しいですね', '仲間由紀恵は怖いですね']
['コミュニケーションは得意です', 'コミュニケーションは難しいですね']
['料理は下手です', '料理は不得手です']
['イタリア料理は得意ですね', 'イタリア料理は大好きですね']
['塩分に気をつけないんですか？', '熱中症に気をつけないんですか？']
['だから荒野というドラマのこれまでの回がNHKで再放送されたら、ぜひ見てみたいです。', '紙の月というドラマのこれまでの回がNHKで再放送されたら、ぜひ見てみたいです。']
['以前にインターネットの通販サイトの広島てっぱんグランプリで送料無料キャンペーンの時に広島の牡蠣も購入したことがあります。', '以前にインターネットの通販サイトのハイチュウで送料無料キャンペーンの時に広島の牡蠣も購入したことがあります。']
['ぼくはくまに参加するアーティストは、宇多田ヒカルさん、ゴッホさん、フェルメールさん、フェルーメルさん、井上陽水さん、椎名林檎さんほか、とても豪華な13組です。', 'Music Key betaに参加するアーティストは、宇多田ヒカルさん、フェルメールさん、ゴッホさん、井上陽水さん、フェルーメルさん、椎名林檎さんほか、とても豪華な13組です。']


In [137]:
score(y, y_pred2)

confusion matrix = 
 [[1308   22]
 [  16   40]]
accuracy =  0.9725829725829725
precision =  0.6451612903225806
recall =  0.7142857142857143
f1 score =  0.6779661016949152


In [138]:
data_path = "../X_y_data/context_content/"
data_name = "repeat.pickle"
dataM = DataManager(data_path)

In [139]:
dataM.save_data(data_name, [y, y_pred2])

success save : ../X_y_data/context_content/repeat.pickle
